# Importando bases

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df_bolsonaro_train = pd.read_excel('Bolsonaro_treino_sem_stop2.xlsx')  
df_lula_train = pd.read_excel('Lula_treino_sem_stop2.xlsx') 
df_simone_train = pd.read_excel('Simone_treino_sem_stop2.xlsx') 
df_ciro_train = pd.read_excel('Ciro_treino_sem_stop2.xlsx') 

In [3]:
df_bolsonaro = pd.read_excel('Bolsonaro_todos_sem_stop.xlsx', index_col=0)
df_lula = pd.read_excel('Lula_todos_sem_stop.xlsx', index_col=0)
df_simone = pd.read_excel('Simone_todos_sem_stop.xlsx', index_col=0)
df_ciro = pd.read_excel('Ciro_todos_sem_stop.xlsx', index_col=0)



In [4]:
df_lula_train = df_lula_train[df_lula_train['Sentimento']!=' ']

In [5]:
print(df_lula_train['Sentimento'].value_counts())
print(df_bolsonaro_train['Sentimento'].value_counts())
print(df_simone_train['Sentimento'].value_counts())
print(df_ciro_train['Sentimento'].value_counts())

p    10335
n     1773
x       81
Name: Sentimento, dtype: int64
p    5065
n     848
x     108
Name: Sentimento, dtype: int64
p    120
x    109
n     50
Name: Sentimento, dtype: int64
p    6807
x     105
n      81
Name: Sentimento, dtype: int64


In [8]:
def train_test_under(df):
    tamanho = df['Sentimento'].value_counts()[2]-5
    
    df_train = df[df['Sentimento']=='p'].sample(n = 100 if len(df[df['Sentimento']=='p'])>100 else len(df[df['Sentimento']=='p']))
    df_train = df_train.append(df[df['Sentimento']=='n'].sample(n = 100 if len(df[df['Sentimento']=='n'])>100 else len(df[df['Sentimento']=='n'])))
    df_train = df_train.append(df[df['Sentimento']=='x'].sample(n = 100 if len(df[df['Sentimento']=='x'])>100 else len(df[df['Sentimento']=='x'])))
    
    df_teste = df.drop(df_train.index)
    
    X_bw_train, bw = Bag_of_words(df_train)
    X_tfidf_train, tfidf = tfidf(df_train)
    Y_train = df_train['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int)
    
    X_bw_teste = Bag_of_words_teste(df_teste, bw )
    X_tfidf_teste = tfidf_teste(df_teste, tfidf)
    Y_teste = df_teste['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int)
    
    return X_bw_train, X_tfidf_train, Y_train, X_bw_teste, X_tfidf_teste, Y_teste, bw, tfidf, df_train, df_teste


In [9]:
# Bag Of Words

def Bag_of_words(df):
    matrix = CountVectorizer()
    text = []
    for i in df.index:
        text.append("".join(df['novo_texto'][i]))
    X = matrix.fit_transform(text)
    return X, matrix

def Bag_of_words_teste(df, matrix):
    text = []
    for i in df.index:
        text.append("".join(df['novo_texto'][i]))
    X = matrix.transform(text)
    return X

def tfidf_(df):
    matrix = TfidfVectorizer()
    text = []
    for i in df.index:
        text.append("".join(df['novo_texto'][i]))
    X = matrix.fit_transform(text)
    return X, matrix

def tfidf_teste(df, matrix):
    text = []
    for i in df.index:
        text.append("".join(df['novo_texto'][i]))
    X = matrix.transform(text)
    return X

def train_test_under_bw(df):
    tamanho = df['Sentimento'].value_counts()[2]-2
    
    df_train = df[df['Sentimento']=='p'].sample(n = tamanho)
    df_train = df_train.append(df[df['Sentimento']=='n'].sample(n = tamanho))
    df_train = df_train.append(df[df['Sentimento']=='x'].sample(n = tamanho))
    
    df_teste = df.drop(df_train.index)
    
    X_bw_train, bw = Bag_of_words(df_train)
    Y_train = df_train['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int)
    
    X_bw_teste = Bag_of_words_teste(df_teste, bw )
    
    Y_teste = df_teste['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int)
    
    return X_bw_train, Y_train, X_bw_teste, Y_teste, bw, df_train, df_teste

def train_test_under_tfidf(df):
    tamanho = df['Sentimento'].value_counts()[2]-2
    
    df_train = df[df['Sentimento']=='p'].sample(n = tamanho)
    df_train = df_train.append(df[df['Sentimento']=='n'].sample(n = tamanho))
    df_train = df_train.append(df[df['Sentimento']=='x'].sample(n = tamanho))
    
    df_teste = df.drop(df_train.index)
    
    X_tfidf_train, tfidf = tfidf_(df_train)
    Y_train = df_train['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int)
    
    X_tfidf_teste = tfidf_teste(df_teste, tfidf)
    Y_teste = df_teste['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int)
    
    return X_tfidf_train, Y_train, X_tfidf_teste, Y_teste, tfidf, df_train, df_teste

def train_model_bw(model, df_train_tudo, num_tests):
    X_train, Y_train, X_teste, Y_teste, bw, df_train, df_teste = train_test_under_bw(df_train_tudo)
    
    model2 = model.fit(X_train.toarray(), Y_train)
    predict = model2.predict(X_teste.toarray())
    accuracy = accuracy_score(Y_teste, predict)
    precision = precision_score(Y_teste, predict, average='macro')
    recall = recall_score(Y_teste, predict, average='macro')
    f1 = f1_score(Y_teste, predict, average='macro')
    score = accuracy + precision + recall + f1
    
    matrix_confusion = confusion_matrix(Y_teste, predict,labels=[-1, 0, 1])
    
    errados = matrix_confusion[0][2]+ matrix_confusion[2][0]
    
    model2_fim, bw_fim, df_train_fim, df_teste_fim, score_fim, errados_fim = model2, bw, df_train, df_teste, score, errados
    for i in range(0,num_tests):
        X_train, Y_train, X_teste, Y_teste, bw, df_train, df_teste = train_test_under_bw(df_train_tudo)

        model2 = model.fit(X_train.toarray(), Y_train)
        predict = model2.predict(X_teste.toarray())
        accuracy = accuracy_score(Y_teste, predict)
        precision = precision_score(Y_teste, predict, average='macro')
        recall = recall_score(Y_teste, predict, average='macro')
        f1 = f1_score(Y_teste, predict, average='macro')
        score = accuracy + precision + recall + f1

        matrix_confusion = confusion_matrix(Y_teste, predict,labels=[-1, 0, 1])
        accuracy_fim, precision_fim,recall_fim, f1_fim, matrix_confusion_fim = accuracy, precision,recall,f1,matrix_confusion 
        errados = matrix_confusion[0][2]+ matrix_confusion[2][0]
        
        if((score_fim<score) & (errados_fim>errados)):
            accuracy_fim, precision_fim,recall_fim, f1_fim, matrix_confusion_fim = accuracy, precision,recall,f1,matrix_confusion 
            model2_fim, bw_fim, df_train_fim, df_teste_fim, score_fim, errados_fim = model2, bw, df_train, df_teste, score, errados
    
    print('accuracy:')
    print(accuracy_fim)
    print('precision:')
    print(precision_fim)
    print('recall:')
    print(recall_fim)
    print('f1:')
    print(f1_fim)
    print('matrix_confusion:')
    print(matrix_confusion_fim)
    print(score_fim/4)
    print(errados_fim)
    return model2_fim, bw_fim, df_train_fim, df_teste_fim     
    
def train_model_tfidf(model, df_train_tudo, num_tests):
    X_train, Y_train, X_teste, Y_teste, tfidf, df_train, df_teste = train_test_under_tfidf(df_train_tudo)
    
    model2 = model.fit(X_train.toarray(), Y_train)
    predict = model2.predict(X_teste.toarray())
    accuracy = accuracy_score(Y_teste, predict)
    precision = precision_score(Y_teste, predict, average='macro')
    recall = recall_score(Y_teste, predict, average='macro')
    f1 = f1_score(Y_teste, predict, average='macro')
    score = accuracy + precision + recall + f1
    
    matrix_confusion = confusion_matrix(Y_teste, predict,labels=[-1, 0, 1])
    accuracy_fim, precision_fim,recall_fim, f1_fim, matrix_confusion_fim = accuracy, precision,recall,f1,matrix_confusion 
    errados = matrix_confusion[0][2]+ matrix_confusion[2][0]
    
    model2_fim, tfidf_fim, df_train_fim, df_teste_fim, score_fim, errados_fim = model2, tfidf, df_train, df_teste, score, errados
    for i in range(0,num_tests):
        X_train, Y_train, X_teste, Y_teste, tfidf, df_train, df_teste = train_test_under_tfidf(df_train_tudo)

        model2 = model.fit(X_train.toarray(), Y_train)
        predict = model2.predict(X_teste.toarray())
        accuracy = accuracy_score(Y_teste, predict)
        precision = precision_score(Y_teste, predict, average='macro')
        recall = recall_score(Y_teste, predict, average='macro')
        f1 = f1_score(Y_teste, predict, average='macro')
        score = accuracy + precision + recall + f1

        matrix_confusion = confusion_matrix(Y_teste, predict,labels=[-1, 0, 1])

        errados = matrix_confusion[0][2]+ matrix_confusion[2][0]
        
        if((score_fim<score) & (errados_fim>errados)):
            accuracy_fim, precision_fim,recall_fim, f1_fim, matrix_confusion_fim = accuracy, precision,recall,f1,matrix_confusion 
            model2_fim, tfidf_fim, df_train_fim, df_teste_fim, score_fim, errados_fim = model2, tfidf, df_train, df_teste, score, errados
    
    print('accuracy:')
    print(accuracy_fim)
    print('precision:')
    print(precision_fim)
    print('recall:')
    print(recall_fim)
    print('f1:')
    print(f1_fim)
    print('matrix_confusion:')
    print(matrix_confusion_fim)
    print(score_fim/4)
    print(errados_fim)
    return model2_fim, tfidf_fim, df_train_fim, df_teste_fim  

def AnaliseSentimentos_bw(df, bw,model, name_colum):
    i1=0
    for i in range(5000,len(df),5000):
        if (i==5000):
            predict = model.predict(Bag_of_words_teste(df[i1:i], bw).toarray())
        else:
            predict = np.append(predict, model.predict(Bag_of_words_teste(df[i1:i], bw).toarray()))
        i1=i
        
    predict = np.append(predict,model.predict(Bag_of_words_teste(df[i1:], bw).toarray()))

    df[name_colum] = predict

    print(np.unique(predict, return_counts=True))
    return df

def AnaliseSentimentos_tfidf(df, tfidf,model, name_colum):
    i1=0
    for i in range(5000,len(df),5000):
        if (i==5000):
            predict = model.predict(tfidf_teste(df[i1:i], tfidf).toarray())
        else:
            predict = np.append(predict, model.predict(tfidf_teste(df[i1:i], tfidf).toarray()))
        i1=i
        
    predict = np.append(predict,model.predict(tfidf_teste(df[i1:], tfidf).toarray()))

    df[name_colum] = predict

    print(np.unique(predict, return_counts=True))
    return df

## Bolsonaro

### Bolsonaro - Arvore de Decisao - bw

In [10]:
%%time
model_tree_bolsonaro_bw, tree_bolsonaro_bw, df_train_tree_bolsonaro_bw, df_teste_tree_bolsonaro_bw = \
train_model_bw(tree.DecisionTreeClassifier(), df_bolsonaro_train, 100)
model_tree_bolsonaro_bw = model_tree_bolsonaro_bw.fit(Bag_of_words_teste(df_train_tree_bolsonaro_bw, tree_bolsonaro_bw), df_train_tree_bolsonaro_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_bolsonaro = AnaliseSentimentos_bw(df_bolsonaro, tree_bolsonaro_bw,model_tree_bolsonaro_bw, 'predict_bw_ArvoreDeDecisao')


accuracy:
0.44590566368577944
precision:
0.37039661180267075
recall:
0.6680198109674533
f1:
0.28841203289751166
matrix_confusion:
[[ 429  145  168]
 [   0    2    0]
 [1915  932 2112]]
0.4649125283752099
964
(array([-1,  0,  1]), array([176753, 566058, 203627], dtype=int64))
Wall time: 1min 9s


### Bolsonaro - Arvore de Decisao - tfidf

In [11]:
%%time
model_tree_bolsonaro_tfidf, tree_bolsonaro_tfidf, df_train_tree_bolsonaro_tfidf, df_teste_tree_bolsonaro_tfidf = \
train_model_tfidf(tree.DecisionTreeClassifier(), df_bolsonaro_train, 100)
model_tree_bolsonaro_tfidf = model_tree_bolsonaro_tfidf.fit(tfidf_teste(df_train_tree_bolsonaro_tfidf, tree_bolsonaro_tfidf), df_train_tree_bolsonaro_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_bolsonaro = AnaliseSentimentos_tfidf(df_bolsonaro, tree_bolsonaro_tfidf,model_tree_bolsonaro_tfidf, 'predict_tfidf_ArvoreDeDecisao')


accuracy:
0.5107837980010521
precision:
0.38446726004504894
recall:
0.6298573731371732
f1:
0.320791703780359
matrix_confusion:
[[ 264  224  254]
 [   0    2    0]
 [ 837 1475 2647]]
0.4614750337409083
1091
(array([-1,  0,  1]), array([140483, 268677, 537278], dtype=int64))
Wall time: 1min 9s


### Bolsonaro - Naive Bayes - bw

In [12]:
%%time
model_nb_bolsonaro_bw, nb_bolsonaro_bw, df_train_nb_bolsonaro_bw, df_teste_nb_bolsonaro_bw = \
train_model_bw(GaussianNB(), df_bolsonaro_train, 100)
model_nb_bolsonaro_bw = model_nb_bolsonaro_bw.fit(Bag_of_words_teste(df_train_nb_bolsonaro_bw, nb_bolsonaro_bw).toarray(), df_train_nb_bolsonaro_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_bolsonaro = AnaliseSentimentos_bw(df_bolsonaro, nb_bolsonaro_bw,model_nb_bolsonaro_bw, 'predict_bw_NaiveBayes')


accuracy:
0.4127652112923023
precision:
0.374894826811166
recall:
0.43905379004512657
f1:
0.2823245458381201
matrix_confusion:
[[ 299  252  191]
 [   0    1    1]
 [1127 1778 2054]]
0.43741775449331477
1214
(array([-1,  0,  1]), array([199068, 379546, 367824], dtype=int64))
Wall time: 2min 29s


### Bolsonaro - Naive Bayes - tfidf

In [13]:
%%time
model_nb_bolsonaro_tfidf, nb_bolsonaro_tfidf, df_train_nb_bolsonaro_tfidf, df_teste_nb_bolsonaro_tfidf = \
train_model_tfidf(GaussianNB(), df_bolsonaro_train, 100)
model_nb_bolsonaro_tfidf = model_nb_bolsonaro_tfidf.fit(tfidf_teste(df_train_nb_bolsonaro_tfidf, nb_bolsonaro_tfidf).toarray(), df_train_nb_bolsonaro_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_bolsonaro = AnaliseSentimentos_tfidf(df_bolsonaro, nb_bolsonaro_tfidf,model_nb_bolsonaro_tfidf, 'predict_tfidf_NaiveBayes')


accuracy:
0.4359109240750482
precision:
0.3750968092824715
recall:
0.630570770162593
f1:
0.2919719021273797
matrix_confusion:
[[ 341  205  196]
 [   0    2    0]
 [1299 1517 2143]]
0.4333876014118731
1495
(array([-1,  0,  1]), array([274968, 337531, 333939], dtype=int64))
Wall time: 2min 37s


### Bolsonaro - Random Forest - bw

In [14]:
%%time
model_RandomForest_bolsonaro_bw, RandomForest_bolsonaro_bw, df_train_RandomForest_bolsonaro_bw, df_teste_RandomForest_bolsonaro_bw = \
train_model_bw(RandomForestClassifier(), df_bolsonaro_train, 100)
model_RandomForest_bolsonaro_bw = model_RandomForest_bolsonaro_bw.fit(Bag_of_words_teste(df_train_RandomForest_bolsonaro_bw, RandomForest_bolsonaro_bw).toarray(), df_train_RandomForest_bolsonaro_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_bolsonaro = AnaliseSentimentos_bw(df_bolsonaro, RandomForest_bolsonaro_bw,model_RandomForest_bolsonaro_bw, 'predict_bw_RandomForest')


accuracy:
0.552691565842539
precision:
0.38621775700655636
recall:
0.48390721254810565
f1:
0.3348229925952249
matrix_confusion:
[[ 276  205  261]
 [   0    1    1]
 [ 867 1217 2875]]
0.45029679892244046
620
(array([-1,  0,  1]), array([ 73288, 440919, 432231], dtype=int64))
Wall time: 3min 9s


### Bolsonaro - Random Forest - tfidf

In [15]:
%%time
model_RandomForest_bolsonaro_tfidf, RandomForest_bolsonaro_tfidf, df_train_RandomForest_bolsonaro_tfidf, df_teste_RandomForest_bolsonaro_tfidf = \
train_model_tfidf(RandomForestClassifier(), df_bolsonaro_train, 100)
model_RandomForest_bolsonaro_tfidf = model_RandomForest_bolsonaro_tfidf.fit(tfidf_teste(df_train_RandomForest_bolsonaro_tfidf, RandomForest_bolsonaro_tfidf).toarray(), df_train_RandomForest_bolsonaro_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_bolsonaro = AnaliseSentimentos_tfidf(df_bolsonaro, RandomForest_bolsonaro_tfidf,model_RandomForest_bolsonaro_tfidf, 'predict_tfidf_RandomForest')


accuracy:
0.6121339645800455
precision:
0.41328776717105836
recall:
0.6629790155284111
f1:
0.3649898665138873
matrix_confusion:
[[ 249  184  309]
 [   0    2    0]
 [ 516 1203 3240]]
0.5133476534483505
825
(array([-1,  0,  1]), array([108363, 267745, 570330], dtype=int64))
Wall time: 2min 59s


### Bolsonaro - SVM - bw

In [16]:
%%time
model_svc_bolsonaro_bw, svc_bolsonaro_bw, df_train_svc_bolsonaro_bw, df_teste_svc_bolsonaro_bw = \
train_model_bw(LinearSVC(), df_bolsonaro_train, 100)
model_svc_bolsonaro_bw = model_svc_bolsonaro_bw.fit(Bag_of_words_teste(df_train_svc_bolsonaro_bw, svc_bolsonaro_bw).toarray(), df_train_svc_bolsonaro_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_bolsonaro = AnaliseSentimentos_bw(df_bolsonaro, svc_bolsonaro_bw,model_svc_bolsonaro_bw, 'predict_bw_SVM')

accuracy:
0.4271436086270384
precision:
0.38300056496085455
recall:
0.28713464786813414
f1:
0.29258883141523057
matrix_confusion:
[[ 323  255  164]
 [   0    0    2]
 [1133 1713 2113]]
0.46580128641476765
1100
(array([-1,  0,  1]), array([196438, 327871, 422129], dtype=int64))
Wall time: 1min 12s


### Bolsonaro - SVM - tfidf

In [17]:
%%time
model_svc_bolsonaro_tfidf, svc_bolsonaro_tfidf, df_train_svc_bolsonaro_tfidf, df_teste_svc_bolsonaro_tfidf = \
train_model_tfidf(LinearSVC(), df_bolsonaro_train, 100)
model_svc_bolsonaro_tfidf = model_svc_bolsonaro_tfidf.fit(tfidf_teste(df_train_svc_bolsonaro_tfidf, svc_bolsonaro_tfidf).toarray(), df_train_svc_bolsonaro_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_bolsonaro = AnaliseSentimentos_tfidf(df_bolsonaro, svc_bolsonaro_tfidf,model_svc_bolsonaro_tfidf, 'predict_tfidf_SVM')

accuracy:
0.500964404699281
precision:
0.3857865556511702
recall:
0.6524524460866618
f1:
0.3214254788927342
matrix_confusion:
[[ 333  203  206]
 [   0    2    0]
 [1105 1332 2522]]
0.4651572213324618
1311
(array([-1,  0,  1]), array([269554, 315450, 361434], dtype=int64))
Wall time: 1min 6s


### Bolsonaro - MLP - bw

In [18]:
%%time
model_mlp_bolsonaro_bw, mlp_bolsonaro_bw, df_train_mlp_bolsonaro_bw, df_teste_mlp_bolsonaro_bw = \
train_model_bw(MLPClassifier(max_iter=400), df_bolsonaro_train, 100)
model_mlp_bolsonaro_bw = model_mlp_bolsonaro_bw.fit(Bag_of_words_teste(df_train_mlp_bolsonaro_bw, mlp_bolsonaro_bw).toarray(), df_train_mlp_bolsonaro_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_bolsonaro = AnaliseSentimentos_bw(df_bolsonaro, mlp_bolsonaro_bw,model_mlp_bolsonaro_bw, 'predict_bw_MLP')

accuracy:
0.4613361388742767
precision:
0.37069397169176904
recall:
0.4740999284881762
f1:
0.2964189242064403
matrix_confusion:
[[ 342  192  208]
 [   0    1    1]
 [1411 1260 2288]]
0.46517446634730036
1205
(array([-1,  0,  1]), array([214506, 242439, 489493], dtype=int64))
Wall time: 8min 44s


### Bolsonaro - MLP - tfidf

In [19]:
%%time
model_mlp_bolsonaro_tfidf, mlp_bolsonaro_tfidf, df_train_mlp_bolsonaro_tfidf, df_teste_mlp_bolsonaro_tfidf = \
train_model_tfidf(MLPClassifier(max_iter=400), df_bolsonaro_train, 100)
model_mlp_bolsonaro_tfidf = model_mlp_bolsonaro_tfidf.fit(Bag_of_words_teste(df_train_mlp_bolsonaro_tfidf, mlp_bolsonaro_tfidf).toarray(), df_train_mlp_bolsonaro_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_bolsonaro = AnaliseSentimentos_tfidf(df_bolsonaro, mlp_bolsonaro_tfidf,model_mlp_bolsonaro_tfidf, 'predict_tfidf_MLP')

accuracy:
0.5048220234964054
precision:
0.37187039925433746
recall:
0.6527851835183274
f1:
0.31257868106287917
matrix_confusion:
[[ 330  168  244]
 [   0    2    0]
 [1308 1104 2547]]
0.46051407183298737
1552
(array([-1,  0,  1]), array([288061, 291102, 367275], dtype=int64))
Wall time: 10min 28s


### Bolsonaro - KNN - bw

In [20]:
%%time
model_knn_bolsonaro_bw, knn_bolsonaro_bw, df_train_knn_bolsonaro_bw, df_teste_knn_bolsonaro_bw = \
train_model_bw(KNeighborsClassifier(), df_bolsonaro_train, 100)
model_knn_bolsonaro_bw = model_knn_bolsonaro_bw.fit(Bag_of_words_teste(df_train_knn_bolsonaro_bw, knn_bolsonaro_bw).toarray(), df_train_knn_bolsonaro_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_bolsonaro = AnaliseSentimentos_bw(df_bolsonaro, knn_bolsonaro_bw,model_knn_bolsonaro_bw, 'predict_bw_KNN')

accuracy:
0.6742065579519552
precision:
0.3932087004234825
recall:
0.28442989929823476
f1:
0.3192570842964406
matrix_confusion:
[[  68  174  500]
 [   0    0    2]
 [ 161 1021 3777]]
0.4511856951907376
672
(array([-1,  0,  1]), array([ 37310, 356657, 552471], dtype=int64))
Wall time: 2min 34s


### Bolsonaro - KNN - tfidf

In [21]:
%%time
model_knn_bolsonaro_tfidf, knn_bolsonaro_tfidf, df_train_knn_bolsonaro_tfidf, df_teste_knn_bolsonaro_tfidf = \
train_model_tfidf(KNeighborsClassifier(), df_bolsonaro_train, 100)
model_knn_bolsonaro_tfidf = model_knn_bolsonaro_tfidf.fit(Bag_of_words_teste(df_train_knn_bolsonaro_tfidf, knn_bolsonaro_tfidf).toarray(), df_train_knn_bolsonaro_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_bolsonaro = AnaliseSentimentos_tfidf(df_bolsonaro, knn_bolsonaro_tfidf,model_knn_bolsonaro_tfidf, 'predict_tfidf_KNN')

accuracy:
0.4215325267403121
precision:
0.36296790621854075
recall:
0.6246768877662964
f1:
0.27624947268585937
matrix_confusion:
[[ 340  200  202]
 [   0    2    0]
 [1583 1314 2062]]
0.42135669835275213
1785
(array([-1,  0,  1]), array([334820, 290879, 320739], dtype=int64))
Wall time: 3min 3s


# Lula

### Lula - Arvore de Decisao - bw

In [22]:
%%time
model_tree_lula_bw, tree_lula_bw, df_train_tree_lula_bw, df_teste_tree_lula_bw = \
train_model_bw(tree.DecisionTreeClassifier(), df_lula_train, 100)
model_tree_lula_bw = model_tree_lula_bw.fit(Bag_of_words_teste(df_train_tree_lula_bw, tree_lula_bw), df_train_tree_lula_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_lula = AnaliseSentimentos_bw(df_lula, tree_lula_bw,model_tree_lula_bw, 'predict_bw_ArvoreDeDecisao')


accuracy:
0.40227576974564927
precision:
0.3827280028738686
recall:
0.5953255072351654
f1:
0.2834150033154175
matrix_confusion:
[[ 644  677  373]
 [   0    2    0]
 [2156 3938 4162]]
0.4618333264303018
1570
(array([-1,  0,  1]), array([188532, 507794, 235404], dtype=int64))
Wall time: 1min 20s


### Lula - Arvore de Decisao - tfidf

In [23]:
%%time
model_tree_lula_tfidf, tree_lula_tfidf, df_train_tree_lula_tfidf, df_teste_tree_lula_tfidf = \
train_model_tfidf(tree.DecisionTreeClassifier(), df_lula_train, 100)
model_tree_lula_tfidf = model_tree_lula_tfidf.fit(tfidf_teste(df_train_tree_lula_tfidf, tree_lula_tfidf), df_train_tree_lula_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_lula = AnaliseSentimentos_tfidf(df_lula, tree_lula_tfidf,model_tree_lula_tfidf, 'predict_tfidf_ArvoreDeDecisao')


accuracy:
0.4861111111111111
precision:
0.4219633335495761
recall:
0.6219780314235002
f1:
0.3342973023136259
matrix_confusion:
[[ 608  716  370]
 [   0    2    0]
 [1224 3832 5200]]
0.46608744459945334
1594
(array([-1,  0,  1]), array([116364, 440987, 374379], dtype=int64))
Wall time: 1min 21s


### Lula - Naive Bayes - bw

In [24]:
%%time
model_nb_lula_bw, nb_lula_bw, df_train_nb_lula_bw, df_teste_nb_lula_bw = \
train_model_bw(GaussianNB(), df_lula_train, 100)
model_nb_lula_bw = model_nb_lula_bw.fit(Bag_of_words_teste(df_train_nb_lula_bw, nb_lula_bw).toarray(), df_train_nb_lula_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_lula = AnaliseSentimentos_bw(df_lula, nb_lula_bw,model_nb_lula_bw, 'predict_bw_NaiveBayes')


accuracy:
0.3660475234270415
precision:
0.36288725411132844
recall:
0.6320123761266861
f1:
0.2517086683031656
matrix_confusion:
[[ 953  422  319]
 [   0    2    0]
 [4548 2288 3420]]
0.35228564797767625
2190
(array([-1,  0,  1]), array([162573, 377950, 391207], dtype=int64))
Wall time: 2min 57s


### Lula - Naive Bayes - tfidf

In [25]:
%%time
model_nb_lula_tfidf, nb_lula_tfidf, df_train_nb_lula_tfidf, df_teste_nb_lula_tfidf = \
train_model_tfidf(GaussianNB(), df_lula_train, 100)
model_nb_lula_tfidf = model_nb_lula_tfidf.fit(tfidf_teste(df_train_nb_lula_tfidf, nb_lula_tfidf).toarray(), df_train_nb_lula_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_lula = AnaliseSentimentos_tfidf(df_lula, nb_lula_tfidf,model_nb_lula_tfidf, 'predict_tfidf_NaiveBayes')


accuracy:
0.47523427041499333
precision:
0.3724717759329013
recall:
0.6192313070326827
f1:
0.30349298235478345
matrix_confusion:
[[ 617  514  563]
 [   0    2    0]
 [2227 2968 5061]]
0.4426075839338402
2790
(array([-1,  0,  1]), array([233038, 345357, 353335], dtype=int64))
Wall time: 2min 50s


### Lula - Random Forest - bw

In [26]:
%%time
model_RandomForest_lula_bw, RandomForest_lula_bw, df_train_RandomForest_lula_bw, df_teste_RandomForest_lula_bw = \
train_model_bw(RandomForestClassifier(), df_lula_train, 100)
model_RandomForest_lula_bw = model_RandomForest_lula_bw.fit(Bag_of_words_teste(df_train_RandomForest_lula_bw, RandomForest_lula_bw).toarray(), df_train_RandomForest_lula_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_lula = AnaliseSentimentos_bw(df_lula, RandomForest_lula_bw,model_RandomForest_lula_bw, 'predict_bw_RandomForest')


accuracy:
0.5005856760374833
precision:
0.4193028359524107
recall:
0.6248081387246044
f1:
0.3371489911029766
matrix_confusion:
[[ 591  671  432]
 [   0    2    0]
 [1191 3675 5390]]
0.4648934697301917
1016
(array([-1,  0,  1]), array([ 80661, 593990, 257079], dtype=int64))
Wall time: 3min 26s


### Lula - Random Forest - tfidf

In [27]:
%%time
model_RandomForest_lula_tfidf, RandomForest_lula_tfidf, df_train_RandomForest_lula_tfidf, df_teste_RandomForest_lula_tfidf = \
train_model_tfidf(RandomForestClassifier(), df_lula_train, 100)
model_RandomForest_lula_tfidf = model_RandomForest_lula_tfidf.fit(tfidf_teste(df_train_RandomForest_lula_tfidf, RandomForest_lula_tfidf).toarray(), df_train_RandomForest_lula_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_lula = AnaliseSentimentos_tfidf(df_lula, RandomForest_lula_tfidf,model_RandomForest_lula_tfidf, 'predict_tfidf_RandomForest')


accuracy:
0.49146586345381527
precision:
0.42925973979717796
recall:
0.6156802618031522
f1:
0.3367941468468553
matrix_confusion:
[[ 557  691  446]
 [   0    2    0]
 [ 970 3971 5315]]
0.4683000029752502
1416
(array([-1,  0,  1]), array([ 95142, 525740, 310848], dtype=int64))
Wall time: 3min 15s


### lula - SVM - bw

In [28]:
%%time
model_svc_lula_bw, svc_lula_bw, df_train_svc_lula_bw, df_teste_svc_lula_bw = \
train_model_bw(LinearSVC(), df_lula_train, 100)
model_svc_lula_bw = model_svc_lula_bw.fit(Bag_of_words_teste(df_train_svc_lula_bw, svc_lula_bw).toarray(), df_train_svc_lula_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_lula = AnaliseSentimentos_bw(df_lula, svc_lula_bw,model_svc_lula_bw, 'predict_bw_SVM')

accuracy:
0.47908299866131193
precision:
0.39700631592601815
recall:
0.47380479634769807
f1:
0.32292441974382813
matrix_confusion:
[[ 737  544  413]
 [   1    1    0]
 [2020 3248 4988]]
0.441872029584391
1770
(array([-1,  0,  1]), array([126088, 437907, 367735], dtype=int64))
Wall time: 1min 26s


### lula - SVM - tfidf

In [29]:
%%time
model_svc_lula_tfidf, svc_lula_tfidf, df_train_svc_lula_tfidf, df_teste_svc_lula_tfidf = \
train_model_tfidf(LinearSVC(), df_lula_train, 100)
model_svc_lula_tfidf = model_svc_lula_tfidf.fit(tfidf_teste(df_train_svc_lula_tfidf, svc_lula_tfidf).toarray(), df_train_svc_lula_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_lula = AnaliseSentimentos_tfidf(df_lula, svc_lula_tfidf,model_svc_lula_tfidf, 'predict_tfidf_SVM')

accuracy:
0.5615796519410977
precision:
0.40451490620362485
recall:
0.6678856380937646
f1:
0.35393387818943234
matrix_confusion:
[[ 709  453  532]
 [   0    2    0]
 [1700 2555 6001]]
0.49697851860697984
2232
(array([-1,  0,  1]), array([237805, 338519, 355406], dtype=int64))
Wall time: 1min 22s


### Lula - MLP - bw

In [30]:
%%time
model_mlp_lula_bw, mlp_lula_bw, df_train_mlp_lula_bw, df_teste_mlp_lula_bw = \
train_model_bw(MLPClassifier(max_iter=400), df_lula_train, 100)
model_mlp_lula_bw = model_mlp_lula_bw.fit(Bag_of_words_teste(df_train_mlp_lula_bw, mlp_lula_bw).toarray(), df_train_mlp_lula_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_lula = AnaliseSentimentos_bw(df_lula, mlp_lula_bw,model_mlp_lula_bw, 'predict_bw_MLP')

accuracy:
0.5460174029451138
precision:
0.37628028584039486
recall:
0.3073810260556821
f1:
0.3269840747749863
matrix_confusion:
[[ 580  430  684]
 [   0    0    2]
 [1917 2393 5946]]
0.4639451784660978
2212
(array([-1,  0,  1]), array([197709, 391916, 342105], dtype=int64))
Wall time: 8min 13s


### Lula - MLP - tfidf

In [31]:
%%time
model_mlp_lula_tfidf, mlp_lula_tfidf, df_train_mlp_lula_tfidf, df_teste_mlp_lula_tfidf = \
train_model_tfidf(MLPClassifier(max_iter=400), df_lula_train, 100)
model_mlp_lula_tfidf = model_mlp_lula_tfidf.fit(Bag_of_words_teste(df_train_mlp_lula_tfidf, mlp_lula_tfidf).toarray(), df_train_mlp_lula_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_lula = AnaliseSentimentos_tfidf(df_lula, mlp_lula_tfidf,model_mlp_lula_tfidf, 'predict_tfidf_MLP')

accuracy:
0.5305388219544847
precision:
0.3807459128579376
recall:
0.6292156526875007
f1:
0.3234493949316066
matrix_confusion:
[[ 547  529  618]
 [   0    2    0]
 [1751 2713 5792]]
0.4659874456078824
2369
(array([-1,  0,  1]), array([198405, 377058, 356267], dtype=int64))
Wall time: 9min 6s


### Lula - KNN - bw

In [32]:
%%time
model_knn_lula_bw, knn_lula_bw, df_train_knn_lula_bw, df_teste_knn_lula_bw = \
train_model_bw(KNeighborsClassifier(), df_lula_train, 100)
model_knn_lula_bw = model_knn_lula_bw.fit(Bag_of_words_teste(df_train_knn_lula_bw, knn_lula_bw).toarray(), df_train_knn_lula_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_lula = AnaliseSentimentos_bw(df_lula, knn_lula_bw,model_knn_lula_bw, 'predict_bw_KNN')

accuracy:
0.24548192771084337
precision:
0.4094869041390881
recall:
0.5009066596430091
f1:
0.2172676130861757
matrix_confusion:
[[ 440 1153  101]
 [   0    2    0]
 [1207 6557 2492]]
0.3391346737190689
421
(array([-1,  0,  1]), array([ 12197, 780423, 139110], dtype=int64))
Wall time: 3min 3s


### Lula - KNN - tfidf

In [33]:
%%time
model_knn_lula_tfidf, knn_lula_tfidf, df_train_knn_lula_tfidf, df_teste_knn_lula_tfidf = \
train_model_tfidf(KNeighborsClassifier(), df_lula_train, 100)
model_knn_lula_tfidf = model_knn_lula_tfidf.fit(Bag_of_words_teste(df_train_knn_lula_tfidf, knn_lula_tfidf).toarray(), df_train_knn_lula_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_lula = AnaliseSentimentos_tfidf(df_lula, knn_lula_tfidf,model_knn_lula_tfidf, 'predict_tfidf_KNN')

accuracy:
0.5133868808567604
precision:
0.38634688422812813
recall:
0.6701916571349985
f1:
0.32806721615864604
matrix_confusion:
[[ 837  400  457]
 [   0    2    0]
 [2684 2275 5297]]
0.47449815959463326
3141
(array([-1,  0,  1]), array([313093, 321141, 297496], dtype=int64))
Wall time: 3min 30s


# Simone

### Simone - Arvore de Decisao - bw

In [34]:
%%time
model_tree_simone_bw, tree_simone_bw, df_train_tree_simone_bw, df_teste_tree_simone_bw = \
train_model_bw(tree.DecisionTreeClassifier(), df_simone_train, 100)
model_tree_simone_bw = model_tree_simone_bw.fit(Bag_of_words_teste(df_train_tree_simone_bw, tree_simone_bw), df_train_tree_simone_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_simone = AnaliseSentimentos_bw(df_simone, tree_simone_bw,model_tree_simone_bw, 'predict_bw_ArvoreDeDecisao')


accuracy:
0.6592592592592592
precision:
0.5184090909090909
recall:
0.6024590163934426
f1:
0.4978080120937263
matrix_confusion:
[[ 1  0  1]
 [ 7 34 20]
 [ 8 10 54]]
0.6044439019068559
8
(array([-1,  0,  1]), array([30830, 27466, 72866], dtype=int64))
Wall time: 6.51 s


### Simone - Arvore de Decisao - tfidf

In [35]:
%%time
model_tree_simone_tfidf, tree_simone_tfidf, df_train_tree_simone_tfidf, df_teste_tree_simone_tfidf = \
train_model_tfidf(tree.DecisionTreeClassifier(), df_simone_train, 100)
model_tree_simone_tfidf = model_tree_simone_tfidf.fit(tfidf_teste(df_train_tree_simone_tfidf, tree_simone_tfidf), df_train_tree_simone_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_simone = AnaliseSentimentos_tfidf(df_simone, tree_simone_tfidf,model_tree_simone_tfidf, 'predict_tfidf_ArvoreDeDecisao')


accuracy:
0.6518518518518519
precision:
0.5077864326857074
recall:
0.5961596842744384
f1:
0.4892191441518566
matrix_confusion:
[[ 1  1  0]
 [11 32 18]
 [ 5 12 55]]
0.5612542782409636
5
(array([-1,  0,  1]), array([44885, 33341, 52936], dtype=int64))
Wall time: 6.76 s


### Simone - Naive Bayes - bw

In [36]:
%%time
model_nb_simone_bw, nb_simone_bw, df_train_nb_simone_bw, df_teste_nb_simone_bw = \
train_model_bw(GaussianNB(), df_simone_train, 100)
model_nb_simone_bw = model_nb_simone_bw.fit(Bag_of_words_teste(df_train_nb_simone_bw, nb_simone_bw).toarray(), df_train_nb_simone_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_simone = AnaliseSentimentos_bw(df_simone, nb_simone_bw,model_nb_simone_bw, 'predict_bw_NaiveBayes')


accuracy:
0.6148148148148148
precision:
0.5076733319101496
recall:
0.5838646023072253
f1:
0.47204481792717085
matrix_confusion:
[[ 1  1  0]
 [ 6 45 10]
 [16 19 37]]
0.6326567394286764
9
(array([-1,  0,  1]), array([39451, 63121, 28590], dtype=int64))
Wall time: 9.64 s


### Simone - Naive Bayes - tfidf

In [37]:
%%time
model_nb_simone_tfidf, nb_simone_tfidf, df_train_nb_simone_tfidf, df_teste_nb_simone_tfidf = \
train_model_tfidf(GaussianNB(), df_simone_train, 100)
model_nb_simone_tfidf = model_nb_simone_tfidf.fit(tfidf_teste(df_train_nb_simone_tfidf, nb_simone_tfidf).toarray(), df_train_nb_simone_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_simone = AnaliseSentimentos_tfidf(df_simone, nb_simone_tfidf,model_nb_simone_tfidf, 'predict_tfidf_NaiveBayes')


accuracy:
0.6518518518518519
precision:
0.5322174883343478
recall:
0.7648755312689738
f1:
0.5233799106540662
matrix_confusion:
[[ 2  0  0]
 [ 5 40 16]
 [13 13 46]]
0.6180811955273099
13
(array([-1,  0,  1]), array([40355, 44625, 46182], dtype=int64))
Wall time: 10 s


### Simone - Random Forest - bw

In [38]:
%%time
model_RandomForest_simone_bw, RandomForest_simone_bw, df_train_RandomForest_simone_bw, df_teste_RandomForest_simone_bw = \
train_model_bw(RandomForestClassifier(), df_simone_train, 100)
model_RandomForest_simone_bw = model_RandomForest_simone_bw.fit(Bag_of_words_teste(df_train_RandomForest_simone_bw, RandomForest_simone_bw).toarray(), df_train_RandomForest_simone_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_simone = AnaliseSentimentos_bw(df_simone, RandomForest_simone_bw,model_RandomForest_simone_bw, 'predict_bw_RandomForest')


accuracy:
0.6666666666666666
precision:
0.5172098690222358
recall:
0.4483910139647844
f1:
0.4797035099193372
matrix_confusion:
[[ 0  1  1]
 [ 9 38 14]
 [10 10 52]]
0.598746830894797
3
(array([-1,  0,  1]), array([18148, 86846, 26168], dtype=int64))
Wall time: 29.1 s


### Simone - Random Forest - tfidf

In [39]:
%%time
model_RandomForest_simone_tfidf, RandomForest_simone_tfidf, df_train_RandomForest_simone_tfidf, df_teste_RandomForest_simone_tfidf = \
train_model_tfidf(RandomForestClassifier(), df_simone_train, 100)
model_RandomForest_simone_tfidf = model_RandomForest_simone_tfidf.fit(tfidf_teste(df_train_RandomForest_simone_tfidf, RandomForest_simone_tfidf).toarray(), df_train_RandomForest_simone_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_simone = AnaliseSentimentos_tfidf(df_simone, RandomForest_simone_tfidf,model_RandomForest_simone_tfidf, 'predict_tfidf_RandomForest')


accuracy:
0.674074074074074
precision:
0.5428388746803069
recall:
0.7745901639344263
f1:
0.5382439744220364
matrix_confusion:
[[ 2  0  0]
 [ 8 35 18]
 [ 7 11 54]]
0.632436771777711
7
(array([-1,  0,  1]), array([33003, 45036, 53123], dtype=int64))
Wall time: 28.3 s


### Simone - SVM - bw

In [40]:
%%time
model_svc_simone_bw, svc_simone_bw, df_train_svc_simone_bw, df_teste_svc_simone_bw = \
train_model_bw(LinearSVC(), df_simone_train, 100)
model_svc_simone_bw = model_svc_simone_bw.fit(Bag_of_words_teste(df_train_svc_simone_bw, svc_simone_bw).toarray(), df_train_svc_simone_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_simone = AnaliseSentimentos_bw(df_simone, svc_simone_bw,model_svc_simone_bw, 'predict_bw_SVM')

accuracy:
0.6222222222222222
precision:
0.5445175438596491
recall:
0.7488615664845173
f1:
0.5012711864406779
matrix_confusion:
[[ 2  0  0]
 [ 9 43  9]
 [19 14 39]]
0.6357774267204993
12
(array([-1,  0,  1]), array([36372, 61135, 33655], dtype=int64))
Wall time: 6.9 s


### Simone - SVM - tfidf

In [41]:
%%time
model_svc_simone_tfidf, svc_simone_tfidf, df_train_svc_simone_tfidf, df_teste_svc_simone_tfidf = \
train_model_tfidf(LinearSVC(), df_simone_train, 100)
model_svc_simone_tfidf = model_svc_simone_tfidf.fit(tfidf_teste(df_train_svc_simone_tfidf, svc_simone_tfidf).toarray(), df_train_svc_simone_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_simone = AnaliseSentimentos_tfidf(df_simone, svc_simone_tfidf,model_svc_simone_tfidf, 'predict_tfidf_SVM')

accuracy:
0.6592592592592592
precision:
0.5461210317460318
recall:
0.7736794171220401
f1:
0.527479211214065
matrix_confusion:
[[ 2  0  0]
 [ 8 45  8]
 [11 19 42]]
0.626634729835349
11
(array([-1,  0,  1]), array([46179, 55322, 29661], dtype=int64))
Wall time: 6.91 s


### Simone - MLP - bw

In [42]:
%%time
model_mlp_simone_bw, mlp_simone_bw, df_train_mlp_simone_bw, df_teste_mlp_simone_bw = \
train_model_bw(MLPClassifier(max_iter=400), df_simone_train, 100)
model_mlp_simone_bw = model_mlp_simone_bw.fit(Bag_of_words_teste(df_train_mlp_simone_bw, mlp_simone_bw).toarray(), df_train_mlp_simone_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_simone = AnaliseSentimentos_bw(df_simone, mlp_simone_bw,model_mlp_simone_bw, 'predict_bw_MLP')

accuracy:
0.6592592592592592
precision:
0.5544168597594615
recall:
0.7720097146326655
f1:
0.5282348852320292
matrix_confusion:
[[ 2  0  0]
 [ 5 43 13]
 [18 10 44]]
0.6173116381232224
5
(array([-1,  0,  1]), array([39545, 73187, 18430], dtype=int64))
Wall time: 1min 38s


### Simone - MLP - tfidf

In [43]:
%%time
model_mlp_simone_tfidf, mlp_simone_tfidf, df_train_mlp_simone_tfidf, df_teste_mlp_simone_tfidf = \
train_model_tfidf(MLPClassifier(max_iter=400), df_simone_train, 100)
model_mlp_simone_tfidf = model_mlp_simone_tfidf.fit(Bag_of_words_teste(df_train_mlp_simone_tfidf, mlp_simone_tfidf).toarray(), df_train_mlp_simone_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_simone = AnaliseSentimentos_tfidf(df_simone, mlp_simone_tfidf,model_mlp_simone_tfidf, 'predict_tfidf_MLP')

accuracy:
0.6592592592592592
precision:
0.5483403459560542
recall:
0.7695051608986034
f1:
0.5274044193447178
matrix_confusion:
[[ 2  0  0]
 [ 6 40 15]
 [15 10 47]]
0.6261272963646587
15
(array([-1,  0,  1]), array([44227, 55582, 31353], dtype=int64))
Wall time: 1min 58s


### Simone - KNN - bw

In [44]:
%%time
model_knn_simone_bw, knn_simone_bw, df_train_knn_simone_bw, df_teste_knn_simone_bw = \
train_model_bw(KNeighborsClassifier(), df_simone_train, 100)
model_knn_simone_bw = model_knn_simone_bw.fit(Bag_of_words_teste(df_train_knn_simone_bw, knn_simone_bw).toarray(), df_train_knn_simone_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_simone = AnaliseSentimentos_bw(df_simone, knn_simone_bw,model_knn_simone_bw, 'predict_bw_KNN')

accuracy:
0.6962962962962963
precision:
0.5128434886499402
recall:
0.4610655737704918
f1:
0.4676038737894408
matrix_confusion:
[[ 0  0  2]
 [ 2 31 28]
 [ 4  5 63]]
0.6815179909425388
1
(array([-1,  0,  1]), array([  7210,  10401, 113551], dtype=int64))
Wall time: 14.7 s


### Simone - KNN - tfidf

In [45]:
%%time
model_knn_simone_tfidf, knn_simone_tfidf, df_train_knn_simone_tfidf, df_teste_knn_simone_tfidf = \
train_model_tfidf(KNeighborsClassifier(), df_simone_train, 100)
model_knn_simone_tfidf = model_knn_simone_tfidf.fit(Bag_of_words_teste(df_train_knn_simone_tfidf, knn_simone_tfidf).toarray(), df_train_knn_simone_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_simone = AnaliseSentimentos_tfidf(df_simone, knn_simone_tfidf,model_knn_simone_tfidf, 'predict_tfidf_KNN')

accuracy:
0.6592592592592592
precision:
0.5532679738562091
recall:
0.7728445658773527
f1:
0.5267686243295999
matrix_confusion:
[[ 2  0  0]
 [ 9 44  8]
 [13 16 43]]
0.6280351058306053
13
(array([-1,  0,  1]), array([45047, 60233, 25882], dtype=int64))
Wall time: 30.1 s


# Ciro

### Ciro - Arvore de Decisao - bw

In [46]:
%%time
model_tree_ciro_bw, tree_ciro_bw, df_train_tree_ciro_bw, df_teste_tree_ciro_bw = \
train_model_bw(tree.DecisionTreeClassifier(), df_ciro_train, 100)
model_tree_ciro_bw = model_tree_ciro_bw.fit(Bag_of_words_teste(df_train_tree_ciro_bw, tree_ciro_bw), df_train_tree_ciro_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_ciro = AnaliseSentimentos_bw(df_ciro, tree_ciro_bw,model_tree_ciro_bw, 'predict_bw_ArvoreDeDecisao')


accuracy:
0.3433984606275903
precision:
0.33497233142336463
recall:
0.4731241806152626
f1:
0.1743765066077209
matrix_confusion:
[[   1    1    0]
 [   7   15    4]
 [2002 2422 2304]]
0.3885513147890517
1108
(array([-1,  0,  1]), array([ 78808, 272317, 146941], dtype=int64))
Wall time: 44.3 s


### Ciro - Arvore de Decisao - tfidf

In [47]:
%%time
model_tree_ciro_tfidf, tree_ciro_tfidf, df_train_tree_ciro_tfidf, df_teste_tree_ciro_tfidf = \
train_model_tfidf(tree.DecisionTreeClassifier(), df_ciro_train, 100)
model_tree_ciro_tfidf = model_tree_ciro_tfidf.fit(tfidf_teste(df_train_tree_ciro_tfidf, tree_ciro_tfidf), df_train_tree_ciro_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_ciro = AnaliseSentimentos_tfidf(df_ciro, tree_ciro_tfidf,model_tree_ciro_tfidf, 'predict_tfidf_ArvoreDeDecisao')


accuracy:
0.5313795145056246
precision:
0.33621859807128945
recall:
0.3949701210402756
f1:
0.23772815861054375
matrix_confusion:
[[   0    0    2]
 [   6   17    3]
 [1481 1674 3573]]
0.37507409805693337
1483
(array([-1,  0,  1]), array([146393, 168978, 182695], dtype=int64))
Wall time: 45.3 s


### Ciro - Naive Bayes - bw

In [48]:
%%time
model_nb_ciro_bw, nb_ciro_bw, df_train_nb_ciro_bw, df_teste_nb_ciro_bw = \
train_model_bw(GaussianNB(), df_ciro_train, 100)
model_nb_ciro_bw = model_nb_ciro_bw.fit(Bag_of_words_teste(df_train_nb_ciro_bw, nb_ciro_bw).toarray(), df_train_nb_ciro_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_ciro = AnaliseSentimentos_bw(df_ciro, nb_ciro_bw,model_nb_ciro_bw, 'predict_bw_NaiveBayes')


accuracy:
0.413854351687389
precision:
0.3355519205737458
recall:
0.43285237354797407
f1:
0.2007976910121861
matrix_confusion:
[[   1    1    0]
 [  10   10    6]
 [2770 1173 2785]]
0.4198046648367239
2476
(array([-1,  0,  1]), array([201842, 130277, 165947], dtype=int64))
Wall time: 1min 28s


### Ciro - Naive Bayes - tfidf

In [49]:
%%time
model_nb_ciro_tfidf, nb_ciro_tfidf, df_train_nb_ciro_tfidf, df_teste_nb_ciro_tfidf = \
train_model_tfidf(GaussianNB(), df_ciro_train, 100)
model_nb_ciro_tfidf = model_nb_ciro_tfidf.fit(tfidf_teste(df_train_nb_ciro_tfidf, nb_ciro_tfidf).toarray(), df_train_nb_ciro_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_ciro = AnaliseSentimentos_tfidf(df_ciro, nb_ciro_tfidf,model_nb_ciro_tfidf, 'predict_tfidf_NaiveBayes')


accuracy:
0.5122853759621078
precision:
0.33654034552265705
recall:
0.657958321900058
f1:
0.23280866023361682
matrix_confusion:
[[   2    0    0]
 [  10   12    4]
 [2071 1210 3447]]
0.4348981759046099
2071
(array([-1,  0,  1]), array([165490, 125883, 206693], dtype=int64))
Wall time: 1min 30s


### Ciro - Random Forest - bw

In [50]:
%%time
model_RandomForest_ciro_bw, RandomForest_ciro_bw, df_train_RandomForest_ciro_bw, df_teste_RandomForest_ciro_bw = \
train_model_bw(RandomForestClassifier(), df_ciro_train, 100)
model_RandomForest_ciro_bw = model_RandomForest_ciro_bw.fit(Bag_of_words_teste(df_train_RandomForest_ciro_bw, RandomForest_ciro_bw).toarray(), df_train_RandomForest_ciro_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_ciro = AnaliseSentimentos_bw(df_ciro, RandomForest_ciro_bw,model_RandomForest_ciro_bw, 'predict_bw_RandomForest')


accuracy:
0.35997631734754293
precision:
0.33652466318527735
recall:
0.632519284124516
f1:
0.18317113148585207
matrix_confusion:
[[   2    0    0]
 [   9   14    3]
 [2945 1367 2416]]
0.4177788489764787
1502
(array([-1,  0,  1]), array([124226, 131196, 242644], dtype=int64))
Wall time: 2min 2s


### Ciro - Random Forest - tfidf

In [51]:
%%time
model_RandomForest_ciro_tfidf, RandomForest_ciro_tfidf, df_train_RandomForest_ciro_tfidf, df_teste_RandomForest_ciro_tfidf = \
train_model_tfidf(RandomForestClassifier(), df_ciro_train, 100)
model_RandomForest_ciro_tfidf = model_RandomForest_ciro_tfidf.fit(tfidf_teste(df_train_RandomForest_ciro_tfidf, RandomForest_ciro_tfidf).toarray(), df_train_RandomForest_ciro_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_ciro = AnaliseSentimentos_tfidf(df_ciro, RandomForest_ciro_tfidf,model_RandomForest_ciro_tfidf, 'predict_tfidf_RandomForest')


accuracy:
0.4502664298401421
precision:
0.3365872382141816
recall:
0.5727499314003476
f1:
0.21360171040477946
matrix_confusion:
[[   1    0    1]
 [   4   20    2]
 [1764 1943 3021]]
0.3933013274648626
1765
(array([-1,  0,  1]), array([141856, 181433, 174777], dtype=int64))
Wall time: 1min 55s


### Ciro - SVM - bw

In [52]:
%%time
model_svc_ciro_bw, svc_ciro_bw, df_train_svc_ciro_bw, df_teste_svc_ciro_bw = \
train_model_bw(LinearSVC(), df_ciro_train, 100)
model_svc_ciro_bw = model_svc_ciro_bw.fit(Bag_of_words_teste(df_train_svc_ciro_bw, svc_ciro_bw).toarray(), df_train_svc_ciro_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_ciro = AnaliseSentimentos_bw(df_ciro, svc_ciro_bw,model_svc_ciro_bw, 'predict_bw_SVM')

accuracy:
0.4012729425695678
precision:
0.33640447400230605
recall:
0.6718840513430288
f1:
0.19751287595852893
matrix_confusion:
[[   2    0    0]
 [   6   16    4]
 [2431 1604 2693]]
0.435422421383787
1863
(array([-1,  0,  1]), array([163916, 160173, 173977], dtype=int64))
Wall time: 49.5 s


### Ciro - SVM - tfidf

In [53]:
%%time
model_svc_ciro_tfidf, svc_ciro_tfidf, df_train_svc_ciro_tfidf, df_teste_svc_ciro_tfidf = \
train_model_tfidf(LinearSVC(), df_ciro_train, 100)
model_svc_ciro_tfidf = model_svc_ciro_tfidf.fit(tfidf_teste(df_train_svc_ciro_tfidf, svc_ciro_tfidf).toarray(), df_train_svc_ciro_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_ciro = AnaliseSentimentos_tfidf(df_ciro, svc_ciro_tfidf,model_svc_ciro_tfidf, 'predict_tfidf_SVM')

accuracy:
0.5011841326228538
precision:
0.335487488400968
recall:
0.6542425073935182
f1:
0.22836595718944072
matrix_confusion:
[[   2    0    0]
 [   5   12    9]
 [1880 1476 3372]]
0.4298200214016952
1880
(array([-1,  0,  1]), array([149420, 178749, 169897], dtype=int64))
Wall time: 47.1 s


### Ciro - MLP - bw

In [54]:
%%time
model_mlp_ciro_bw, mlp_ciro_bw, df_train_mlp_ciro_bw, df_teste_mlp_ciro_bw = \
train_model_bw(MLPClassifier(max_iter=400), df_ciro_train, 100)
model_mlp_ciro_bw = model_mlp_ciro_bw.fit(Bag_of_words_teste(df_train_mlp_ciro_bw, mlp_ciro_bw).toarray(), df_train_mlp_ciro_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_ciro = AnaliseSentimentos_bw(df_ciro, mlp_ciro_bw,model_mlp_ciro_bw, 'predict_bw_MLP')

accuracy:
0.4628478389579633
precision:
0.33575069433301236
recall:
0.4875644074514467
f1:
0.21659080409522946
matrix_confusion:
[[   1    0    1]
 [   9   13    4]
 [2076 1539 3113]]
0.4542035830540992
1771
(array([-1,  0,  1]), array([137573, 143982, 216511], dtype=int64))
Wall time: 4min 16s


### Ciro - MLP - tfidf

In [55]:
%%time
model_mlp_ciro_tfidf, mlp_ciro_tfidf, df_train_mlp_ciro_tfidf, df_teste_mlp_ciro_tfidf = \
train_model_tfidf(MLPClassifier(max_iter=400), df_ciro_train, 100)
model_mlp_ciro_tfidf = model_mlp_ciro_tfidf.fit(Bag_of_words_teste(df_train_mlp_ciro_tfidf, mlp_ciro_tfidf).toarray(), df_train_mlp_ciro_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_ciro = AnaliseSentimentos_tfidf(df_ciro, mlp_ciro_tfidf,model_mlp_ciro_tfidf, 'predict_tfidf_MLP')

accuracy:
0.48134991119005327
precision:
0.3366889346941444
recall:
0.7242293972377206
f1:
0.2240360362261498
matrix_confusion:
[[   2    0    0]
 [   2   18    6]
 [1854 1642 3232]]
0.44157606983701697
1854
(array([-1,  0,  1]), array([143289, 165278, 189499], dtype=int64))
Wall time: 4min 51s


### Ciro - KNN - bw

In [56]:
%%time
model_knn_ciro_bw, knn_ciro_bw, df_train_knn_ciro_bw, df_teste_knn_ciro_bw = \
train_model_bw(KNeighborsClassifier(), df_ciro_train, 100)
model_knn_ciro_bw = model_knn_ciro_bw.fit(Bag_of_words_teste(df_train_knn_ciro_bw, knn_ciro_bw).toarray(), df_train_knn_ciro_bw['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_ciro = AnaliseSentimentos_bw(df_ciro, knn_ciro_bw,model_knn_ciro_bw, 'predict_bw_KNN')

accuracy:
0.06867969212551805
precision:
0.33504988735114255
recall:
0.2273240037806031
f1:
0.04502483304576033
matrix_confusion:
[[   0    2    0]
 [  10   16    0]
 [3191 3089  448]]
0.41747804763194934
1471
(array([-1,  0,  1]), array([110245,  36176, 351645], dtype=int64))
Wall time: 1min 47s


### Ciro - KNN - tfidf

In [57]:
%%time
model_knn_ciro_tfidf, knn_ciro_tfidf, df_train_knn_ciro_tfidf, df_teste_knn_ciro_tfidf = \
train_model_tfidf(KNeighborsClassifier(), df_ciro_train, 100)
model_knn_ciro_tfidf = model_knn_ciro_tfidf.fit(Bag_of_words_teste(df_train_knn_ciro_tfidf, knn_ciro_tfidf).toarray(), df_train_knn_ciro_tfidf['Sentimento'].str.replace('p','1').replace('x','0').replace('n','-1').astype(int))

df_ciro = AnaliseSentimentos_tfidf(df_ciro, knn_ciro_tfidf,model_knn_ciro_tfidf, 'predict_tfidf_KNN')

accuracy:
0.436056838365897
precision:
0.33653341785321755
recall:
0.6707559681697614
f1:
0.20990331544341878
matrix_confusion:
[[   2    0    0]
 [   4   15    7]
 [2472 1327 2929]]
0.41331238495807365
2472
(array([-1,  0,  1]), array([192657, 148566, 156843], dtype=int64))
Wall time: 2min 3s


In [58]:
df_bolsonaro.to_excel('bolsonaro_predict4.xlsx')
df_lula.to_excel('lula_predict4.xlsx')
df_simone.to_excel('simone_predict4.xlsx')
df_ciro.to_excel('ciro_predict4.xlsx')

In [59]:
def VotacaoPredict(df):
    for i in df.index:
        pos=0
        neu=0
        neg=0
        for j in ['predict_tfidf_MLP','predict_bw_MLP',
                  'predict_tfidf_SVM','predict_bw_SVM',
                 'predict_tfidf_RandomForest','predict_bw_RandomForest',
                 'predict_tfidf_NaiveBayes','predict_bw_NaiveBayes',
                 'predict_tfidf_ArvoreDeDecisao','predict_bw_ArvoreDeDecisao',
                 'predict_tfidf_KNN', 'predict_bw_KNN']:
            if (df.loc[i,j]==1):
                pos+=1
            elif (df.loc[i,j]==0):
                neu+=1    
            else:
                neg+=1
        
        if((neu>=neg)& (neu>=pos)):
            df.loc[i,'VotacaoPredict_norm'] = 0
        elif((neg>=pos)& (neg>=neu)):
            df.loc[i,'VotacaoPredict_norm'] = -1
        else:
            df.loc[i,'VotacaoPredict_norm'] = 1     
    #print(df['VotacaoPredict_ajust'].value_counts())
    print(df['VotacaoPredict_norm'].value_counts())        
    return df

In [60]:
df_bolsonaro_predict2 = VotacaoPredict(df_bolsonaro)
df_lula_predict2 = VotacaoPredict(df_lula)
df_simone_predict2 = VotacaoPredict(df_simone)
df_ciro_predict2 = VotacaoPredict(df_ciro)

 1.0    457670
 0.0    385905
-1.0    102863
Name: VotacaoPredict_norm, dtype: int64
 0.0    568775
 1.0    291736
-1.0     71219
Name: VotacaoPredict_norm, dtype: int64
 0.0    61299
-1.0    41702
 1.0    28161
Name: VotacaoPredict_norm, dtype: int64
 1.0    183238
 0.0    172200
-1.0    142628
Name: VotacaoPredict_norm, dtype: int64


In [61]:
df_bolsonaro_predict2.to_excel('bolsonaro_resultado_equilibrio2.xlsx')
df_lula_predict2.to_excel('lula_resultado_equilibrio2.xlsx')
df_simone_predict2.to_excel('simone_resultado_equilibrio2.xlsx')
df_ciro_predict2.to_excel('ciro_resultado_equilibrio2.xlsx')